In [1]:
import os
import copy
import torch
import pickle
import pprint
import random
import shutil
import random
import torch
import hashlib
import numpy as np
import pandas as pd
from rnn import RNN
import torch.nn as nn
from vocab import Vocab
from torch import optim
from util import *
from constants import *
from evaluator import Evaluator
from tqdm.notebook import tqdm
from sklearn.metrics import *
from transformers import AdamW
from collections import Counter
import torch.nn.functional as F
from collections import OrderedDict
from torch.autograd import Variable
from sklearn.preprocessing import *
from embedding_layer import EmbeddingLayer
from attention_layer import AttentionLayer
from torch.nn.utils.rnn import pad_sequence
from gensim.models.word2vec import Word2Vec
from dataloaders import TextDataLoader, TextDataset
from gensim.models.keyedvectors import KeyedVectors
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
tqdm.pandas()

2022-12-01 15:26:50.607827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 15:26:50.788577: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!nvidia-smi

Thu Dec  1 15:26:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   55C    P0    58W / 300W |  32032MiB / 32508MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   

In [3]:
device

device(type='cuda', index=3)

In [4]:
def read_data(file_path) -> list:
    data = pd.read_csv(file_path, on_bad_lines='skip')
    id_data = data[id_col_name]
    text_data = data[text_col_name]
    hierarchical_label_data = []
    label_data = data[label_col_name].tolist()
    output = []
    for i in tqdm(range(len(label_data)), desc="Reading data"):
        labels = label_data[i].split(';')
        output.append((text_data[i], labels, id_data[i]))
    return output

In [5]:
def read_labeldesc(file_path) -> list:
    data = pd.read_csv(file_path)
    desc_data = data[desc_col_name].tolist()
    label_data = data[label_col_name].tolist()
    output = []
    for i in tqdm(range(len(label_data)), desc="Reading Label Descriptions"):
        output.append((label_data[i], desc_data[i]))
    return output

In [6]:
def load_cached_data(file_path: str) -> tuple:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        f.close()
    return data["vocab"], data["training_data"], data["valid_data"], data["test_data"],data["label_desc"]

In [7]:
def prepare_data():
    cache_folder = "{}/{}".format(cache_dir, "mimic3_single_50")
#     device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
    save_vocab_file_name = "{}.pkl".format("save_vocab")
    cached_file_name = os.path.join(cache_folder, save_vocab_file_name)
    if os.path.exists(cached_file_name):
        vocab, training_data, valid_data, test_data, label_desc = load_cached_data(cached_file_name)
        data = training_data + valid_data + test_data
        labels = []
        for (feature, l, _) in data:
            labels.extend(l)
        unique_labels=list(set(labels))
        n_labels = len(unique_labels)
        labels = unique_labels
        vocab.update_labels(labels)
        print("Loaded vocab and data from file")
    else:
        training_data = read_data( data_dir + "/train.csv")
        valid_data = read_data( data_dir + "/dev.csv")
        test_data = read_data( data_dir + "/test.csv")
        label_desc = read_labeldesc(data_dir + "/D_ICD_50.csv")
        data = training_data + valid_data + test_data
        labels = []
        for (feature, l, _) in data:
            labels.extend(l)
        unique_labels=list(set(labels))
        n_labels = len(unique_labels)
        labels = unique_labels
        vocab = Vocab(data, labels,label_desc,min_word_frequency,
                          word_embedding_mode=embedding_mode,
                          word_embedding_file=embedding_file)
        print("Preparing the vocab")
        vocab.prepare_vocab()
        saved_objects = {"vocab": vocab,
                         "training_data": training_data,
                         "valid_data": valid_data,
                         "test_data": test_data,
                         "label_desc": label_desc}
        with open(cached_file_name, 'wb') as f:
            pickle.dump(saved_objects, f, pickle.HIGHEST_PROTOCOL)
            f.close()
        print("Saved vocab and data to files")
    return training_data, valid_data, test_data ,label_desc, vocab,cached_file_name

In [8]:
training_data, valid_data, test_data, label_desc, vocab, saved_vocab_path = prepare_data()

Loaded vocab and data from file


In [9]:
model = RNN(vocab, device)
model

RNN(
  (embedding): EmbeddingLayer(
    (embeddings): Embedding(66379, 100)
  )
  (label_linear): Linear(in_features=512, out_features=256, bias=True)
  (rnn): LSTM(100, 256, bidirectional=True)
  (rnn1): LSTM(100, 256, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (attention): AttentionLayer(
    (first_linear): Linear(in_features=512, out_features=256, bias=False)
    (second_linear): Linear(in_features=256, out_features=50, bias=False)
    (third_linear): Linear(in_features=512, out_features=256, bias=False)
  )
  (classification_layer): Linear(in_features=256, out_features=50, bias=True)
)

In [10]:
class Trainer:
    def __init__(self, model: nn.Module,
                 train_dataloader: TextDataLoader,
                 valid_dataloader: TextDataLoader,
                 test_dataloader: TextDataLoader,
                 criterions,
                 optimiser,
                 scheduler,
                 vocab,
                 checkpoint_path,
                 ):
        self.model = model
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.test_dataloader = test_dataloader
        self.criterions = criterions
        self.optimiser = optimiser
        self.scheduler = scheduler
        self.vocab = vocab
        self.multilabel = multilabel
        self.checkpoint_path = checkpoint_path
        self.n_training_labels = n_labels
        self.saved_result_path = '../model/results.pkl'
        self.main_metric = main_metric
        self.start_epoch = 0
        self.best_val = None
        self.saved_test_scores = None
        self.best_epoch_num = 1

    def train_single_epoch(self, index):
        self.model.train()
        self.train_dataloader.dataset.shuffle_data()
        losses = []
        true_labels = []
        pred_probs = []
        ids = []
        all_loss_list = []
        progress_bar = tqdm(self.train_dataloader, unit="batches", desc="Training at epoch #{}".format(index))
        progress_bar.clear()
        self.optimiser.zero_grad()
        batch_id = 0

        for text_batch, label_batch, length_batch, id_batch,desc_batch in progress_bar:
            batch_id += 1
            text_batch = text_batch.to(device)
            label_batch = label_batch.to(device)
            length_batch = length_batch.to(device)
            desc_batch = desc_batch.to(device)
            true_labels.extend(label_batch.cpu().numpy())
            ids.extend(id_batch)
            output = self.model(text_batch, length_batch,desc_batch)
            loss_list = []
            loss_list = self.criterions(output, label_batch)
            all_loss_list.append([loss_list.item()])
            output = torch.sigmoid(output)
            output = output.detach().cpu().numpy()
            pred_probs.extend(output)
            loss = get_loss(loss_list, self.n_training_labels)
            loss.backward()
            losses.append(loss.item())
            self.optimiser.step()
            self.optimiser.zero_grad()

        scores = OrderedDict()
        scores = calculate_eval_metrics(ids, true_labels,pred_probs, self.multilabel)
        scores["loss"] = np.mean(all_loss_list).item()
        scores["average"] = np.mean(losses).item()
        progress_bar.refresh(True)
        progress_bar.clear(True)
        progress_bar.close()
        keys = ['micro_auc', 'macro_auc', 'micro_f1', 'macro_f1', 'macro_P@5', 'macro_P@8', 'macro_P@15', 'loss']
        print({k:v for k,v in scores.items() if k in keys})
        return scores


    @staticmethod
    def format_number(number):
        return abs(round(number, ndigits=ndigits))

    def train(self,n_epoch: int = 100):
        best_valid_scores = self.best_val
        saved_test_scores = self.saved_test_scores
        saved_train_scores = None
        best_epoch_num = self.best_epoch_num
        evaluator = Evaluator(self.model, self.vocab, self.criterions, self.n_training_labels,device)
        for e in range(self.start_epoch + 1, n_epoch + 1):
            train_scores = self.train_single_epoch(e)
            epoch_loss = train_scores["average"]
            valid_scores = evaluator.evaluate(self.valid_dataloader)
            test_scores = evaluator.evaluate(self.test_dataloader)
            self.scheduler.step(test_scores["average"])
            print('lr {}'.format(self.optimiser.param_groups[0]['lr']))
            keys = ['micro_auc', 'macro_auc', 'micro_f1', 'macro_f1', 'macro_P@5', 'macro_P@8', 'macro_P@15', 'loss']
            print({k:v for k,v in test_scores.items() if k in keys})
            print('\n\n')
            
            if best_valid_scores is None or best_valid_scores[self.main_metric] < valid_scores[self.main_metric]:
                best_valid_scores = valid_scores
                saved_test_scores = test_scores
                saved_train_scores = train_scores
                best_epoch_num = e
                

                    
        results = {"train": saved_train_scores, "valid": best_valid_scores, "test": saved_test_scores,
                    "index2label": self.vocab.index2label}
        with open(self.saved_result_path, 'wb') as f:
            pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)
                
        return self.model, best_valid_scores

In [11]:
def _load_and_cache_data(train_data, valid_data, test_data, label_desc, vocab, saved_data_file_path):
    if os.path.exists(saved_data_file_path):
        try:
            with open(saved_data_file_path, 'rb') as f:
                data = pickle.load(f)
                data["train"].multilabel = bool(multilabel)
                data["valid"].multilabel = bool(multilabel)
                data["test"].multilabel = bool(multilabel)

                return data["train"], data["valid"], data["test"]
        except:
            pass

    # Build train/valid/test data loaders
    train_dataset = TextDataset(train_data, vocab, label_desc,max_seq_length=max_seq_length,min_seq_length=min_seq_length,sort=True,multilabel=multilabel)

    valid_dataset = TextDataset(valid_data, vocab, label_desc,max_seq_length=max_seq_length,min_seq_length=min_seq_length,sort=True,multilabel=multilabel)

    test_dataset = TextDataset(test_data, vocab, label_desc,max_seq_length=max_seq_length,min_seq_length=min_seq_length,sort=True, multilabel=multilabel)
    # try:
    with open(saved_data_file_path, 'wb') as f:
        data = {"train": train_dataset, "valid": valid_dataset, "test": test_dataset}
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    
    return train_dataset, valid_dataset, test_dataset

In [12]:
def _train_model(train_data, valid_data, test_data, label_desc, vocab, saved_data_file_path=None,checkpoint_path=None):
    model = RNN(vocab,device)
    init_state_dict = None
    if init_state_dict is not None:
        model.load_state_dict(init_state_dict)
    model.to(device)

    train_dataset, valid_dataset, test_dataset = _load_and_cache_data(train_data, valid_data, test_data, label_desc,
                                                                      vocab, saved_data_file_path)
    train_dataloader = TextDataLoader(dataset=train_dataset, vocab=vocab, batch_size=batch_size)

    valid_dataloader = TextDataLoader(dataset=valid_dataset, vocab=vocab, batch_size=batch_size)

    test_dataloader = TextDataLoader(dataset=test_dataset, vocab=vocab, batch_size=batch_size)

    optimiser = torch.optim.AdamW(model.parameters(), lr = 0.001)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', patience=1)
    
    criterions = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(2))

    trainer = Trainer(model=model,
                      train_dataloader=train_dataloader,
                      valid_dataloader=valid_dataloader,
                      test_dataloader=test_dataloader,
                      criterions=criterions,
                      optimiser=optimiser,
                      scheduler = scheduler,
                      vocab=vocab,
                      checkpoint_path=checkpoint_path)
    best_model, scores = trainer.train(n_epoch=n_epoch)

    evaluator = Evaluator(model=best_model,vocab=vocab,criterions=criterions,
                          n_training_labels=n_labels,device = device)

    del model, optimiser, evaluator, trainer, criterions
    return best_model, scores  # either on valid or test

In [13]:
def run_with_validation(training_data, valid_data, test_data, label_desc, vocab, saved_data_file_path):
    best_model, scores = _train_model(
        train_data=training_data, valid_data=valid_data, test_data=test_data, label_desc=label_desc,
        vocab=vocab, saved_data_file_path=saved_data_file_path, checkpoint_path="../model/checkpoint.pkl")

    return best_model, scores

In [ ]:
# lstm no third layer
run_with_validation(training_data, valid_data, test_data, label_desc, vocab, saved_data_file_path="{}.data.pkl".format(saved_vocab_path.split(".pkl")[0]))

Training at epoch #1:   0%|          | 0/253 [00:00<?, ?batches/s]

{'macro_f1': 0.24678628248933546, 'macro_auc': 0.714228952278815, 'macro_P@5': 0.39590875278948673, 'macro_P@8': 0.32981031490205803, 'macro_P@15': 0.24530126456731965, 'micro_f1': 0.37228357209448903, 'micro_auc': 0.7871644972275891, 'loss': 0.45516749338905804}


Evaluating:   0%|          | 0/50 [00:00<?, ?batches/s]

Evaluating:   0%|          | 0/55 [00:00<?, ?batches/s]

lr 0.001
{'macro_f1': 0.4291462882146768, 'macro_auc': 0.8262305188299266, 'macro_P@5': 0.508502024291498, 'macro_P@8': 0.4183776749566223, 'macro_P@15': 0.3016001542317332, 'micro_f1': 0.5263751518306438, 'micro_auc': 0.8672122834775144, 'loss': 0.3949410246177153}





...

{'macro_f1': 0.7454315418300281, 'macro_auc': 0.9603038071312483, 'macro_P@5': 0.7046119514009421, 'macro_P@8': 0.5660953384577238, 'macro_P@15': 0.35936854285478137, 'micro_f1': 0.7757951421422762, 'micro_auc': 0.9728808188395051, 'loss': 0.18519707636635294}


Evaluating:   0%|          | 0/50 [00:00<?, ?batches/s]

Evaluating:   0%|          | 0/55 [00:00<?, ?batches/s]

lr 1.0000000000000002e-06
{'macro_f1': 0.6802894220536543, 'macro_auc': 0.9222431598951446, 'macro_P@5': 0.6668594563331406, 'macro_P@8': 0.5443175245806825, 'macro_P@15': 0.35878156930788513, 'micro_f1': 0.7127081663549737, 'micro_auc': 0.9445871977488434, 'loss': 0.27829607637091114}





Training at epoch #16:   0%|          | 0/253 [00:00<?, ?batches/s]